In [824]:
import json
import numpy as np
import os
from pprint import pprint
from scipy.stats import ttest_ind

num_evals = 5
evals = {}

In [856]:
task_dir = '/Users/ethanperez/research/ParlAI/parlai/mturk/core/run_data/live/context_evaluator_'

task_setup = [
# ## Persuading Humans (Dev)
# ('race', 'question', 1554052233, 'No Passage'),
# ('race', 'quote and question', 1554006689, 'TFIDF(S,[Q;A])'),
# ('race', 'quote and question', 1554130485, 'TFIDF(S,A)'),
# ('race', 'quote and question', 1554069931, 'BERT Base'),
# ('race', 'quote and question', 1554072277, 'Pred search'),
# ('race', 'quote and question', 1554132868, 'Pred Δp(A)'),
# ## Persuading Humans
# ('race', 'quote and question', 1556832343, 'Pred search (Almost complete)'),
# ('race', 'quote and question', 1557155351, 'Bi-Ranker'),
# ## Persuading Humans: FINAL
# ('race', 'question', 1557351318, 'No Passage'),
# ('race', 'quote and question', 1556671432, 'TFIDF(S,[Q;A])'),
# ('race', 'quote and question', 1556725767, 'TFIDF(S,A)'),
# ('race', 'quote and question', 1556739336, 'FastText(S,A)'),
# ('race', 'quote and question', 1557144204, 'BERT Base'),
# ('race', 'quote and question', 1556892630, 'Pred search'),
# ('race', 'quote and question', 1556809031, 'Pred p(A)'),
# ('race', 'quote and question', 1556756789, 'Pred Δp(A)'),  # (race.m=sl-sents.i.best.e)
# ('race', 'quote and question', 1557420471, 'Human 1'),
# ('race', 'quote and question', 1558016111, 'Human 2'),
# ('race', 'quote and question', 1558028902, 'Human 3'),
# ## Human QA on Summary
# ('race', 'quotes and question', 1555946909, 'FastText(S,A) (lower pay)'),
# ('race', 'quotes and question', 1555952058, 'BERT Base Best Epoch'),  # (6-10 sentence incorrectly placed at end)
# ('race', 'quotes and question', 1556939750, 'Pred Δp(A) (lower pay)'),
# ('race', 'quotes and question', 1557093902, 'Bi-Ranker'),
# ## Human QA on Summary: FINAL
# ('race', 'passage and question', 1555823963, 'Full Passage'),
# ('race', 'quotes and question', 1557189780, 'TFIDF(S,[Q;A])'),
# ('race', 'quotes and question', 1557164593, 'TFIDF(S,A)'),
# ('race', 'quotes and question', 1557234076, 'FastText(S,A)'),
# ('race', 'quotes and question', 1557085110, 'BERT Base'),  # Last Epoch
# ('race', 'quotes and question', 1556987177, 'Pred search'),
# ('race', 'quotes and question', 1556977072, 'Pred p(A)'),
# ('race', 'quotes and question', 1556999857, 'Pred Δp(A)'),
# ('race', 'quotes and question', 1557432288, 'Human 1'),
('race', 'quotes and question', 1558039471, 'Human 2'),
# ('race', 'quotes and question', ?, 'Human 3'),
# ## Persuading Humans
# ('dream', 'quote and question', 1556670413, 'Bi-Ranker'),
# ('dream', 'quote and question', 1555333992, 'Pred search (ToM)'),
# ## Persuading Humans: FINAL
# ('dream', 'question', 1554582693, 'No Passage'),
# ('dream', 'quote and question', 1554596686, 'TFIDF(S,[Q;A])'),
# ('dream', 'quote and question', 1554587404, 'TFIDF(S,A)'),
# ('dream', 'quote and question', 1554662280, 'FastText(S,A)'),
# ('dream', 'quote and question', 1554675304, 'BERT Base'),
# ('dream', 'quote and question', 1554685131, 'Pred search'),
# ('dream', 'quote and question', 1554692472, 'Pred p(A)'),
# ('dream', 'quote and question', 1554729998, 'Pred Δp(A)'),
# ('dream', 'quote and question', 1558028987, 'Human 1'),
# ('dream', 'quotes and question', ?, 'Human 2'),
# ('dream', 'quotes and question', ?, 'Human 3'),
# ## Human QA on Summary
# ('dream', 'question, answers, and quotes', 1555707929, 'TFIDF(S,A)'),  # 64.21%: (Less filter / no feedback)
# ('dream', 'question, answers, and quotes', 1555722489, 'BERT Base'),  # 65.38%: (Less filter / no feedback)
# ('dream', 'question and quotes', 1555789302, 'Pred search'),  # 75.17% (4/5 filter)
# ('dream', 'question and quotes', 1555812443, 'Pred search'),  # 79.32% Actually: quotes and question (4/5 filter)
# ('dream', 'quotes and question', 1555946647, 'BERT Base (RACE -> DREAM)'),  # (4 sentences incorrectly placed at end) (80.84%)
# ('dream', 'quotes and question', 1556740293, 'Bi-Ranker'),
# ## Human QA on Summary: FINAL
# ('dream', 'passage and question', 1555804551, 'Full Passage'),
# ('dream', 'quotes and question', 1556811067, 'TFIDF(S,[Q;A])'),
# ('dream', 'quotes and question', 1556757043, 'TFIDF(S,A)'),
# ('dream', 'quotes and question', 1555823257, 'FastText(S,A)'),
# ('dream', 'quotes and question', 1556727396, 'BERT Base'),
# ('dream', 'quotes and question', 1556832115, 'Pred search'),
# ('dream', 'quotes and question', 1556892896, 'Pred p(A)'),
# ('dream', 'quotes and question', 1556938429, 'Pred Δp(A)'),
# ('dream', 'quotes and question', 1558016251, 'Human 1'),
# ('dream', 'quotes and question', ?, 'Human 2'),
# ('dream', 'quotes and question', ?, 'Human 3'),
]

dataset, prompt_type, task_id, name = task_setup[-1]
split = None  # 'middle', 'high', None


# Set useful variables
task_dir += str(task_id)
if dataset != 'race':
    split = None
num_options = 3 if dataset == 'dream' else 4
options = ['A', 'B', 'C', 'D'][:num_options]
debate_mode_to_option = {'Ⅰ': 'A', 'Ⅱ': 'B', 'Ⅲ': 'C', 'Ⅳ': 'D', 'ⅰ': 'A', 'ⅱ': 'B', 'ⅲ': 'C', 'ⅳ': 'D', None: None}
question_type_labels = ['a', 'c', 'l', 'm', 's'] if dataset == 'dream' else ['a', 'b', 'c', 'd', 'e']

# if (dataset == 'dream') and (prompt_type == 'quote and question'):
#     question_type_labels = []

# Read HIT data
hit_results = []
num_passed_test = 0
for hit_dir in os.listdir(task_dir):
    if hit_dir.startswith('o_'):
        continue
    num_passed_test += 1
    with open(os.path.join(task_dir, hit_dir, 'custom/data.json'), 'r') as file:
        hit_results.append(json.load(file))
        file.close()
if len(os.listdir(task_dir)) != num_passed_test:
    num_total_tested = len(os.listdir(task_dir)) - num_passed_test
    print('Qualified:', round((100. * num_passed_test) / num_total_tested, 2), '%')

Qualified: 47.5 %


In [857]:
num_valid_hits, num_rejected_hits, num_incomplete_hits = 0, 0, 0
metrics = {}
task_ratings = {i: 0 for i in range(11)}
quote_ratings = []
durations = []
worker_durations = []
accuracy_by_worker = {}
max_response_freqs = []
worker_ids = []
hits_by_qid = {}
debate_mode_counts = {debate_mode: 0 for debate_mode in debate_mode_to_option.keys()}

for hit_result in hit_results:
    if ((len(hit_result['reject_reasons']) > 0) or
        (len(hit_result['block_reasons']) > 0)):
        num_rejected_hits += 1
        print(hit_result['worker_id'], hit_result['assignment_id'],
              '| reject_reasons:', hit_result['reject_reasons'],
              '| block_reasons:', hit_result['block_reasons'],
              '| bonus_reasons: ' + str(hit_result['bonus_reasons']) if 'bonus_reasons' in hit_result else '')
    elif hit_result['feedback'] is None:
        num_incomplete_hits += 1
        continue
    
    worker_ids.append(hit_result['worker_id'])
    num_valid_hits += 1
    if (hit_result['task_rating'] is not None) and (hit_result['task_rating'].isdigit()):
        task_ratings[int(hit_result['task_rating'])] += 1
    if (hit_result.get('quote_rating') is not None) and (hit_result['quote_rating'].isdigit()):
        quote_ratings.append(int(hit_result['quote_rating']))
    for qtype, qtype_accuracy in hit_result['accuracy'].items():
        accuracy_by_worker[qtype] = accuracy_by_worker.get(qtype, []) + [qtype_accuracy]
    
    hit_durations = []
    response_option_counts = {option: 0 for option in options}
    responses = []
    for prompt in hit_result['data']:
        qid = prompt['sample']['qid']
        if (split is not None) and (split not in qid):
            continue
        hits_by_qid[qid] = hits_by_qid.get(qid, [])
        hits_by_qid[qid].append(prompt)
        debate_mode_counts[prompt['sample']['debate_mode']] += 1
        stance = debate_mode_to_option[prompt['sample']['debate_mode']]
        answer = prompt['sample']['eval_labels'][0]
        human_correct = (prompt['response'] == answer)
        assert answer in options, 'Answer must be in options.'
        
        # Calculate metrics
        if qid not in metrics:
            metrics[qid] = {
                option: {
                    'num': 0,
                    'num_correct': 0,
                    'num_correct_debate_mode': 0,
                    'num_incorrect_debate_mode': 0,
                    'num_correct_with_correct_debate_mode': 0,
                    'num_correct_with_incorrect_debate_mode': 0,
                    'num_debate_mode_responses': 0,
                    'is_debate_mode_response': []
                }
                for option in ([None] if stance is None else options)
            }
#             for qtype in question_type_labels:
#                 metrics[qid][qtype] = {
#                     'num': 0,
#                     'num_correct': 0,
#                 }
            metrics[qid]['answer'] = answer
        metrics[qid]['qtype'] = metrics[qid].get('qtype', set([]))
        for qtype in set(''.join(prompt['sample'].get('question_type_labels', []))):
            qtype = qtype.lower()
            metrics[qid]['qtype'].add(qtype)
#             if qtype not in metrics[qid]:
#                 print('Did you set `dataset` appropriately?')
#             metrics[qid][qtype]['num'] += 1
#             metrics[qid][qtype]['num_correct'] += human_correct
        prompt_metrics = metrics[qid][stance]
        prompt_metrics['num'] += 1
        prompt_metrics['num_correct'] += human_correct
        if stance == answer:
            prompt_metrics['num_correct_with_correct_debate_mode'] += human_correct
            prompt_metrics['num_correct_debate_mode'] += 1
        else:
            prompt_metrics['num_correct_with_incorrect_debate_mode'] += human_correct
            prompt_metrics['num_incorrect_debate_mode'] += 1
        prompt_metrics['num_debate_mode_responses'] += (prompt['response'] == stance)
        prompt_metrics['is_debate_mode_response'].append(prompt['response'] == stance)
        
        hit_durations.append(prompt['duration'] / 1000.)
        response_option_counts[prompt['response']] += 1
        responses.append(prompt['response'])
    duration = np.sum(np.array(hit_durations))
    worker_durations.append(duration)
    durations += hit_durations
    response_options_array = np.array(list(response_option_counts.values()))
    response_options_array = response_options_array / response_options_array.sum()
    max_response_freq = response_options_array.max()
    max_response_freqs.append(max_response_freq)
    acc = round(100 * hit_result['accuracy'][prompt_type])
    print('| Time:', round(duration, 1),
          '| Acc:', acc,
          '| Max Freq:', round(100 * max_response_freq, 1),
          '| Rate:', hit_result['task_rating'],
          '| Feedback:', hit_result['feedback'],
          '| Quote Rating:', None if 'quote_rating' not in hit_result else hit_result['quote_rating'], 
          '| Quote Desc:', None if 'quote_description' not in hit_result else hit_result['quote_description'])

debate_modes_used = list(filter(lambda x: debate_mode_counts[x] > 0, debate_mode_counts.keys()))

good_worker_durations = []
assert len(worker_durations) == len(accuracy_by_worker[prompt_type])
for worker_duration, worker_accuracy in zip(worker_durations, accuracy_by_worker[prompt_type]):
    if worker_accuracy > np.median(np.array(accuracy_by_worker[prompt_type])):
        good_worker_durations.append(worker_duration)

quote_ratings = np.array(quote_ratings)
durations = np.array(durations)
worker_durations = np.array(worker_durations)
good_worker_durations = np.array(good_worker_durations)
max_response_freqs = np.array(max_response_freqs)

quote_ratings.sort()
durations.sort()
worker_durations.sort()
good_worker_durations.sort()
max_response_freqs.sort()

for qtype in accuracy_by_worker:
    accuracy_by_worker[qtype] = np.array(accuracy_by_worker[qtype])
    accuracy_by_worker[qtype].sort()
print('REJECTED:', num_rejected_hits)
print('INCOMPLETE:', num_incomplete_hits)
print('VALID:', num_valid_hits)
print('Median Question Duration:', np.median(durations))
print('Mean Question Duration:', np.mean(durations[int(durations.shape[0] / 10.):int(9. * durations.shape[0] / 10.)]))
print('Min/Median/Mean/Max Worker Duration:',
      round(np.min(worker_durations / 60.), 2), '/',
      round(np.median(worker_durations / 60.), 2), '/',
      round(np.mean(worker_durations / 60.), 2), '/',
      round(np.max(worker_durations / 60.), 2))
print('Min/Median/Mean/Max Good Worker Durations:',
      round(np.min(good_worker_durations / 60.), 2), '/',
      round(np.median(good_worker_durations / 60.), 2),'/',
      round(np.mean(good_worker_durations / 60.), 2), '/',
      round(np.max(good_worker_durations / 60.), 2))
print('Median Worker Accuracy:', np.median(accuracy_by_worker[prompt_type]))
print('Median Max Response Freq:', np.median(max_response_freqs))
print('Quote Rating:',
      '| Mean:', round(quote_ratings.mean(), 2),
      '| Median:', round(np.median(quote_ratings), 2),
      '| Std:', round(np.std(quote_ratings), 2))
# pprint(hit_results[0]['data'][0])
# pprint(hit_results[0])
print('debate_mode_counts:', debate_mode_counts)

qids = list(metrics.keys())
qids.sort()

| Time: 726.2 | Acc: 90 | Max Freq: 30.0 | Rate: 10 | Feedback: No comment | Quote Rating: 8 | Quote Desc: Overall useful, one of them was missing vital information needed to answer the question though.
| Time: 919.6 | Acc: 80 | Max Freq: 30.0 | Rate: 0 | Feedback: I can't think of anything at this point. | Quote Rating: 7 | Quote Desc: Fairly helpful.
| Time: 1109.8 | Acc: 85 | Max Freq: 35.0 | Rate: 10 | Feedback: I don't think there needs to be any improvements made. This is a fun and interesting task as it is now.  | Quote Rating: 9 | Quote Desc: Most of them were good and very helpful. There were a few that were a little confusing.
| Time: 1715.9 | Acc: 95 | Max Freq: 30.0 | Rate: 10 | Feedback: Nothing comes to mind | Quote Rating: 10 | Quote Desc: Informative and concise.
| Time: 774.1 | Acc: 75 | Max Freq: 45.0 | Rate: 9 | Feedback: good as is | Quote Rating: 7 | Quote Desc: mixed some were straight forward others were partial or slightly ambiguous 
| Time: 840.3 | Acc: 80 | Ma

In [858]:
def nps(task_ratings):
    num_ratings = sum(list(task_ratings.values()))
    if num_ratings == 0:
        return None

    nps_sum_ratings = 0
    sum_ratings = 0
    for score, num_raters in task_ratings.items():
        sum_ratings += num_raters * score
        if score >= 9:
            nps_sum_ratings += num_raters
        elif score <= 6:
            nps_sum_ratings -= num_raters
    return round(100 * (nps_sum_ratings / float(num_ratings)), 2), round((sum_ratings / float(num_ratings)), 2)

print('NPS, Mean:', nps(task_ratings))

NPS, Mean: (33.33, 8.21)


In [859]:
accuracy_by_qtype = {qtype: [] for qtype in question_type_labels}
accuracy_by_sample = []
accuracy_by_sample_correct_debate_mode = []
accuracy_by_sample_incorrect_debate_mode = []
convinced_by_qtype = {qtype: [] for qtype in question_type_labels}
convinced_freqs = []
convinced_freqs_with_correct_debate_mode = []
convinced_freqs_with_incorrect_debate_mode = []
convinced_freqs_by_qtype = {qtype: [] for qtype in question_type_labels}
convinced_freqs_with_correct_debate_mode_by_qtype = {qtype: [] for qtype in question_type_labels}
convinced_freqs_with_incorrect_debate_mode_by_qtype = {qtype: [] for qtype in question_type_labels}
num_evals_by_sample = []
convinced_by_sample = []
# qtypes = []
# qtypes_with_correct_debate_mode = []
# qtypes_with_incorrect_debate_mode = []

for qid in qids:
    qid_metrics = metrics[qid]
    answer = qid_metrics['answer']
    for qid_metric_key, prompt in qid_metrics.items():
        if qid_metric_key in question_type_labels:
#             qtype = qid_metric_key
#             if qid_metrics[qtype]['num'] > 0:
#                 accuracy_by_qtype[qtype].append(qid_metrics[qtype]['num_correct'] / qid_metrics[qtype]['num'])
#                 convinced_by_qtype[qtype].append(qid_metrics[qtype][])
            continue
        if not (qid_metric_key in [None] + options):
            continue
        stance = qid_metric_key

        # Q-only stats
        prompt_metrics = qid_metrics[stance]
        num_evals_by_sample.append(prompt_metrics['num'])
        convinced_by_sample.append(prompt_metrics['is_debate_mode_response'])
        accuracy_by_sample.append(prompt_metrics['num_correct'] / prompt_metrics['num'])
        for qtype in qid_metrics['qtype']:
            accuracy_by_qtype[qtype].append(prompt_metrics['num_correct'] / prompt_metrics['num'])
        if stance is None:
            continue
        
        # Debater stats
#         if 'num_debate_mode_responses' not in prompt_metrics:
#             print(qid_metric_key, prompt_metrics)
        convinced_freq = prompt_metrics['num_debate_mode_responses'] / prompt_metrics['num']
        if stance == answer:
            convinced_freqs_with_correct_debate_mode.append(convinced_freq)
            for qtype in qid_metrics['qtype']:
                convinced_freqs_with_correct_debate_mode_by_qtype[qtype].append(convinced_freq)
#             qtypes_with_correct_debate_mode.append(qid_metrics['qtype'])
            accuracy_by_sample_correct_debate_mode.append(
                prompt_metrics['num_correct_with_correct_debate_mode'] /
                prompt_metrics['num_correct_debate_mode'])
        else:
            convinced_freqs_with_incorrect_debate_mode.append(convinced_freq)
            for qtype in qid_metrics['qtype']:
                convinced_freqs_with_incorrect_debate_mode_by_qtype[qtype].append(convinced_freq)
#             qtypes_with_incorrect_debate_mode.append(qid_metrics['qtype'])
            accuracy_by_sample_incorrect_debate_mode.append(
                prompt_metrics['num_correct_with_incorrect_debate_mode'] /
                prompt_metrics['num_incorrect_debate_mode'])
        convinced_freqs.append(convinced_freq)
        for qtype in qid_metrics['qtype']:
            convinced_freqs_by_qtype[qtype].append(convinced_freq)
#         qtypes.append(qid_metrics['qtype'])

# qtypes = np.array(qtypes)
# qtypes_with_correct_debate_mode = np.array(qtypes_with_correct_debate_mode)
# qtypes_with_incorrect_debate_mode = np.array(qtypes_with_incorrect_debate_mode)

# print(accuracy_by_qtype)
accuracy_by_qtype = {qtype: (np.array(accuracy_by_qtype[qtype]).mean(), len(accuracy_by_qtype[qtype])) for qtype in question_type_labels}
pprint(accuracy_by_qtype)

num_evals_by_sample = np.array(num_evals_by_sample)
print('Evals per sample:', num_evals_by_sample.mean())
print('Fraction insuffient evals:', (num_evals_by_sample < 5).mean())

convinced_freqs = np.array(convinced_freqs)
print('Convinced:', round(100 * convinced_freqs.mean(), 2), '%')
print('Convinced:', 100 * convinced_freqs.mean(), '%')
convinced_freqs_with_correct_debate_mode = np.array(convinced_freqs_with_correct_debate_mode)
print('- Agent is right:', round(100 * convinced_freqs_with_correct_debate_mode.mean(), 2), '%')
convinced_freqs_with_incorrect_debate_mode = np.array(convinced_freqs_with_incorrect_debate_mode)
print('- Agent is wrong:', round(100 * convinced_freqs_with_incorrect_debate_mode.mean(), 2), '%')

for qtype in question_type_labels:
#     print(len(convinced_freqs_by_qtype[qtype]))
    convinced_freqs_by_qtype[qtype] = np.array(convinced_freqs_by_qtype[qtype]).mean()
    print(len(convinced_freqs_with_correct_debate_mode_by_qtype[qtype]))
    convinced_freqs_with_correct_debate_mode_by_qtype[qtype] = np.array(convinced_freqs_with_correct_debate_mode_by_qtype[qtype]).mean()
#     print(len(convinced_freqs_with_incorrect_debate_mode_by_qtype[qtype]))
    convinced_freqs_with_incorrect_debate_mode_by_qtype[qtype] = np.array(convinced_freqs_with_incorrect_debate_mode_by_qtype[qtype]).mean()

accuracy_by_sample = np.array(accuracy_by_sample)
print('Accuracy:', round(100 * accuracy_by_sample.mean(), 2), '%')
print('Accuracy:', 100 * accuracy_by_sample.mean(), '%')
accuracy_by_sample_correct_debate_mode = np.array(accuracy_by_sample_correct_debate_mode)

num_target_evals = 5
# print('Extra Evals:', round(((100. * (num_evals_by_sample - num_target_evals).sum()) / num_evals_by_sample.sum()), 2), '%')
# num_evals_by_sample.sort()
# print('Evals per sample distribution:', num_evals_by_sample)
# 1.5*3.1*60/(917.5684545454544*26/(20*60))

{'a': (0.7916666666666666, 24),
 'b': (0.7931547619047619, 32),
 'c': (0.7658730158730158, 54),
 'd': (0.7169811320754715, 53),
 'e': (0.6116071428571429, 16)}
Evals per sample: 6.6
Fraction insuffient evals: 0.0
Convinced: nan %
Convinced: nan %
- Agent is right: nan %
- Agent is wrong: nan %
0
0
0
0
0
Accuracy: 74.98 %
Accuracy: 74.97619047619048 %


/anaconda3/envs/main/lib/python3.6/site-packages/ipykernel_launcher.py:80: RuntimeWarning: Mean of empty slice.
/anaconda3/envs/main/lib/python3.6/site-packages/ipykernel_launcher.py:81: RuntimeWarning: Mean of empty slice.
/anaconda3/envs/main/lib/python3.6/site-packages/ipykernel_launcher.py:83: RuntimeWarning: Mean of empty slice.
/anaconda3/envs/main/lib/python3.6/site-packages/ipykernel_launcher.py:85: RuntimeWarning: Mean of empty slice.
/anaconda3/envs/main/lib/python3.6/site-packages/ipykernel_launcher.py:89: RuntimeWarning: Mean of empty slice.
/anaconda3/envs/main/lib/python3.6/site-packages/ipykernel_launcher.py:91: RuntimeWarning: Mean of empty slice.
/anaconda3/envs/main/lib/python3.6/site-packages/ipykernel_launcher.py:93: RuntimeWarning: Mean of empty slice.


In [851]:
evals[name] = []
for eval_no in range(num_evals):
    evals[name].append([convinced_array[eval_no] for convinced_array in convinced_by_sample])

evals[name] = 100. * np.array(evals[name]).mean(axis=1)
pprint(evals)

for n, eval_values in evals.items():
    print('*****', n, '*****')
    print('MEAN:', round(eval_values.mean(), 1))
    print('STD:', round(eval_values.std(), 2))
    print('STDERR:', round(eval_values.std() / np.sqrt(eval_values.shape[0]), 2))
    print()

for n1, eval_values1 in evals.items():
    for n2, eval_values2 in evals.items():
        print(n1, '/', n2, ':', round(ttest_ind(eval_values1, eval_values2, equal_var=False)[1], 4))

{'Human 1': array([48.66666667, 46.33333333, 49.66666667, 50.33333333, 47.        ]),
 'Human 2': array([37.75, 46.25, 45.25, 45.5 , 44.25])}
***** Human 2 *****
MEAN: 43.8
STD: 3.09
STDERR: 1.38

***** Human 1 *****
MEAN: 48.4
STD: 1.53
STDERR: 0.68

Human 2 / Human 2 : 1.0
Human 2 / Human 1 : 0.0381
Human 1 / Human 2 : 0.0381
Human 1 / Human 1 : 1.0


In [817]:
# print('Accuracy/Num-Samples by Q Type:')
if len(accuracy_by_qtype) > 0:
    for qtype in question_type_labels:
        print(round(100. * accuracy_by_qtype[qtype][0], 2))

53.75
89.43
91.79
95.0
96.54


In [818]:
# print(100. * convinced_freqs.mean())
# print(100. * convinced_freqs_with_correct_debate_mode.mean())
# print(100. * convinced_freqs_with_incorrect_debate_mode.mean())

In [797]:
# for qtype in question_type_labels:
#     print(100. * convinced_freqs[qtypes == qtype].mean(), convinced_freqs[qtypes == qtype].sum())
    
# for qtype in question_type_labels:
#     print(100. * convinced_freqs_with_correct_debate_mode[qtypes_with_correct_debate_mode == qtype].mean(), convinced_freqs_with_correct_debate_mode[qtypes_with_correct_debate_mode == qtype].sum())
    
# for qtype in question_type_labels:
#     print(100. * convinced_freqs_with_incorrect_debate_mode[qtypes_with_incorrect_debate_mode == qtype].mean(), convinced_freqs_with_incorrect_debate_mode[qtypes_with_incorrect_debate_mode == qtype].sum())

In [798]:
for qtype in question_type_labels:
    print(100. * convinced_freqs_by_qtype[qtype])
for qtype in question_type_labels:
    print(100. * convinced_freqs_with_correct_debate_mode_by_qtype[qtype])
for qtype in question_type_labels:
    print(100. * convinced_freqs_with_incorrect_debate_mode_by_qtype[qtype])

39.179421768707485
35.60817805383022
38.56632653061225
38.315781440781436
33.431919642857146
80.64625850340137
65.2536231884058
78.39455782312926
69.21855921855924
56.607142857142854
25.357142857142854
25.726363008971703
25.290249433106577
28.014855514855512
25.706845238095237


In [799]:
print(100. * accuracy_by_sample.mean())
for qtype in question_type_labels:
    print(100. * accuracy_by_qtype[qtype][0])

50.43154761904762
54.058248299319736
53.369565217391305
52.782312925170075
51.573565323565326
43.80952380952381


In [800]:
print(100 * convinced_freqs.mean())

37.12414965986394
